In [44]:
import numpy as np

class STN():
    
    """Create a state-task network object.
    
    """
    
    def __init__(self, H=10):
        self.H = H
        self._states = set()
        self._tasks = set()
        self._units = set()
        
    def __setattr__(self, name, value):
        if type(value) == State:
            self._states.add(name)
        elif type(value) == Task:
            self._tasks.add(name)
        elif type(value) == Unit:
            self._units.add(name)
        self.__dict__[name] = value
        
    def __repr__(self):
        s = "States:\n"
        for name in sorted(self._states):
            state = getattr(self, name)
            s += "    {0:20s}".format(str(state))
            s += "    {0:6.1f}".format(state.price)
            s += "    {0:6.1f}".format(state.initialize)
            s += "    {0:6.1f}\n".format(state.capacity)

        s += "\nTasks:\n"
        for name in sorted(self._tasks):
            task = getattr(self, name)
            s += "    {0:12s}\n".format(str(task))
            print(task.feeds)
            for f in task.feeds:
                s += "           Feed: {0:12s}".format(str(f))
                print(s)
                print(getattr(f,'name'))
                s += "  {0:4.2f}".format(getattr(self,f)['rho'])
                s += "\n"
            s += "\n"
            #for f in task.feeds:
            #    s += "\n           Feed: {0:12s}".format(str(f))
            #    s += "  {0:4.2f}".format(self.feeds[f]['rho'])
            #    s += "  {0:4.2f}".format(self.feeds[f]['dur'])
        
        s += "\nUnits:\n"
        for name in sorted(self._units):
            unit = getattr(self, name)
            s += "    {0:12s}\n".format(str(unit))
        return s


class State():
    def __init__(self, name, initialize=0, price=0, capacity=np.Inf):
        self.name = name
        self.initialize = initialize
        self.price = price
        self.capacity = capacity
        
    def __repr__(self):
        return "<State>" + self.name


class Task():
    def __init__(self, name):
        self.name = name
        self.feeds = dict()
        self.products = dict()
        self.units = dict()
        
    def __repr__(self):
        print(self.feeds)
        print(self.products)
        print(self.units)
        return "<Task>" + self.name
        
    #def __str__(self):
        #s = "    {0:12s}".format(self.name)
        #for f in self.feeds:
        #    s += "\n           Feed: {0:12s}".format(str(f))
        #    s += "  {0:4.2f}".format(self.feeds[f]['rho'])
        #    s += "  {0:4.2f}".format(self.feeds[f]['dur'])
        #for p in self.products:
        #    s += "\n        Product: {0:12s}".format(str(p))
        #    s += "  {0:4.2f}".format(self.products[p]['rho'])
        #    s += "  {0:4.2f}".format(self.products[p]['dur'])
        #for u in self.units:
        #    s += "\n           Unit: {0:12s}".format(str(u))
        #    s += "  {0:4.2f}".format(self.units[u]['Bmin'])
        #    s += "  {0:4.2f}".format(self.units[u]['Bmax'])
        #    s += "  {0:4.2f}".format(self.units[u]['tclean'])
        #    s += "  {0:4.2f}".format(self.units[u]['cost'])
        #    s += "  {0:4.2f}".format(self.units[u]['vcost'])
        #return s + "\n"
    
    def feed(self, name, rho=1, dur=0):
        self.feeds[name] = {'rho': rho, 'dur': dur}
    
    def product(self, name, rho=1, dur=0):
        self.products[name] = {'rho': rho, 'dur': dur}
        
    def unit(self, name, Bmin=0, Bmax=np.Inf, tclean=0, cost=0, vcost=0):
        self.units[name] = {
            'Bmin': Bmin, 
            'Bmax': Bmax, 
            'tclean': tclean, 
            'cost': cost, 
            'vcost': vcost}
        name.tasks[self] = {
            'Bmin': Bmin, 
            'Bmax': Bmax, 
            'tclean': tclean, 
            'cost': cost, 
            'vcost': vcost}
        
#class Feed():
#    def __init__(state, task, rho=1, dur=0):
        

class Unit():
    def __init__(self, name):
        self.name = name
        self.tasks = dict()
        
    def __repr__(self):
        return "<Unit>" + self.name
    
    def task(self, name, Bmin=0, Bmax=np.Inf, tclean=0, cost=0, vcost=0):
        self.tasks[name] = {
            'Bmin': Bmin, 
            'Bmax': Bmax, 
            'tclean': tclean, 
            'cost': cost, 
            'vcost': vcost}
        name.units[self] = {
            'Bmin': Bmin, 
            'Bmax': Bmax, 
            'tclean': tclean, 
            'cost': cost, 
            'vcost': vcost}

In [2]:
from stn import Class

ModuleNotFoundError: No module named 'stn'

In [45]:
stn = STN(10)

# specify states
stn.feedA = State('FeedA', initialize=200)
stn.feedB = State('FeedB', initialize=200)
stn.feedC = State('FeedC', initialize=200)
stn.hotA  = State('HotA', capacity=100, price=-1)
stn.intAB = State('IntAB', capacity=200, price=-1)
stn.intBC = State('IntBC', capacity=150, price=-1)
stn.impureE = State('ImpureE', capacity=100, price=-1)
stn.product1 = State('Product_1', price=10)
stn.product2 = State('Product_2', price=10)

# specify tasks
stn.heating = Task('Heating')
stn.heating.feed(stn.feedA)

stn.reaction1 = Task('Reaction 1')
stn.reaction1.feed(stn.feedB, rho=0.5)
stn.reaction1.feed(stn.feedC, rho=0.5)
stn.reaction1.product(stn.product1, rho=0.4, dur=2)
stn.reaction1.product(stn.intAB, rho=0.6, dur=2)

stn.reaction2 = Task('Reaction 2')
stn.reaction2.feed(stn.hotA, rho=0.4)
stn.reaction2.feed(stn.intBC, rho=0.6)

stn.reaction3 = Task('Reaction 3')
stn.reaction3.feed(stn.feedC, rho=0.2)
stn.reaction3.feed(stn.intAB, rho=0.8)
stn.reaction3.product(stn.impureE, dur=1)

stn.separation = Task('Separation')
stn.separation.feed(stn.impureE)
stn.separation.product(stn.product2, rho=0.9, dur=1)
stn.separation.product(stn.intAB, rho=0.1, dur=2)

# specify units
stn.heater = Unit('Heater')
stn.heater.task(stn.heating, Bmax=200)

stn.reactor1 = Unit('Reactor_1')
stn.reactor1.task(stn.reaction1, Bmax=80)
stn.reactor1.task(stn.reaction2, Bmax=80)
stn.reactor1.task(stn.reaction3, Bmax=80)

stn.reactor2 = Unit('Reactor_2')
stn.reactor2.task(stn.reaction1, Bmax=50)
stn.reactor2.task(stn.reaction2, Bmax=50)
stn.reactor2.task(stn.reaction3, Bmax=50)

stn.still = Unit('Still')
stn.still.task(stn.separation, Bmax=200)

In [46]:
stn

{<State>FeedA: {'rho': 1, 'dur': 0}}
{}
{<Unit>Heater: {'Bmin': 0, 'Bmax': 200, 'tclean': 0, 'cost': 0, 'vcost': 0}}
{<State>FeedA: {'rho': 1, 'dur': 0}}
States:
    <State>FeedA               0.0     200.0       inf
    <State>FeedB               0.0     200.0       inf
    <State>FeedC               0.0     200.0       inf
    <State>HotA               -1.0       0.0     100.0
    <State>ImpureE            -1.0       0.0     100.0
    <State>IntAB              -1.0       0.0     200.0
    <State>IntBC              -1.0       0.0     150.0
    <State>Product_1          10.0       0.0       inf
    <State>Product_2          10.0       0.0       inf

Tasks:
    <Task>Heating
           Feed: <State>FeedA
FeedA


TypeError: getattr(): attribute name must be string

In [40]:
stn.reactor1.tasks

{<State>FeedB: {'rho': 0.5, 'dur': 0}, <State>FeedC: {'rho': 0.5, 'dur': 0}}
{<State>Product_1: {'rho': 0.4, 'dur': 2}, <State>IntAB: {'rho': 0.6, 'dur': 2}}
{<Unit>Reactor_1: {'Bmin': 0, 'Bmax': 80, 'tclean': 0, 'cost': 0, 'vcost': 0}, <Unit>Reactor_2: {'Bmin': 0, 'Bmax': 50, 'tclean': 0, 'cost': 0, 'vcost': 0}}
{<State>HotA: {'rho': 0.4, 'dur': 0}, <State>IntBC: {'rho': 0.6, 'dur': 0}}
{}
{<Unit>Reactor_1: {'Bmin': 0, 'Bmax': 80, 'tclean': 0, 'cost': 0, 'vcost': 0}, <Unit>Reactor_2: {'Bmin': 0, 'Bmax': 50, 'tclean': 0, 'cost': 0, 'vcost': 0}}
{<State>FeedC: {'rho': 0.2, 'dur': 0}, <State>IntAB: {'rho': 0.8, 'dur': 0}}
{<State>ImpureE: {'rho': 1, 'dur': 1}}
{<Unit>Reactor_1: {'Bmin': 0, 'Bmax': 80, 'tclean': 0, 'cost': 0, 'vcost': 0}, <Unit>Reactor_2: {'Bmin': 0, 'Bmax': 50, 'tclean': 0, 'cost': 0, 'vcost': 0}}


{<Task>Reaction 1: {'Bmin': 0, 'Bmax': 80, 'tclean': 0, 'cost': 0, 'vcost': 0},
 <Task>Reaction 2: {'Bmin': 0, 'Bmax': 80, 'tclean': 0, 'cost': 0, 'vcost': 0},
 <Task>Reaction 3: {'Bmin': 0, 'Bmax': 80, 'tclean': 0, 'cost': 0, 'vcost': 0}}

In [13]:
stn.reaction1.units

{<Unit>Reactor_1: {'Bmin': 0, 'Bmax': 80, 'tclean': 0, 'cost': 0, 'vcost': 0},
 <Unit>Reactor_2: {'Bmin': 0, 'Bmax': 50, 'tclean': 0, 'cost': 0, 'vcost': 0}}

In [14]:
stn._tasks

{'heating', 'reaction1', 'reaction2', 'reaction3', 'separation'}

In [15]:
for f in stn._tasks:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-15-88195c1bb58e>, line 2)

In [765]:
stn.heating

Heating

In [768]:
task = getattr(stn, 'reaction1')

In [772]:
task.feeds[stn.feedB]['dur']

0